# 🇻🇳 GPT-SoVITS Vietnamese - Voice Cloning Pipeline

Notebook tối ưu để clone giọng nói tiếng Việt trên Google Colab.

**Quy trình:** Upload Audio → Cắt + ASR → Mở WebUI (Train + Inference)

---

In [ ]:
# @title 1️⃣ Cài đặt môi trường
# @markdown Chạy cell này để cài đặt toàn bộ thư viện cần thiết.
# @markdown ⏱️ Thời gian: ~5-7 phút

import os
import sys
from IPython.display import clear_output

# Clone repo (nếu chưa có)
if not os.path.exists("GPT-SoVITS"):
    print("📥 Đang tải GPT-SoVITS...")
    !git clone https://github.com/RVC-Boss/GPT-SoVITS.git

%cd GPT-SoVITS

# Cài đặt dependencies
print("📦 Đang cài đặt thư viện...")
!pip install -q -r requirements.txt
!pip install -q faster-whisper openai-whisper

# Tạo thư mục pretrained models
base_model_dir = "GPT_SoVITS/pretrained_models"
os.makedirs(f"{base_model_dir}/chinese-roberta-wwm-ext-large", exist_ok=True)
os.makedirs(f"{base_model_dir}/chinese-hubert-base", exist_ok=True)

# Tải pretrained models
print("📥 Đang tải pretrained models...")
!wget -nc -q -P {base_model_dir}/chinese-roberta-wwm-ext-large https://huggingface.co/hfl/chinese-roberta-wwm-ext-large/resolve/main/config.json
!wget -nc -q -P {base_model_dir}/chinese-roberta-wwm-ext-large https://huggingface.co/hfl/chinese-roberta-wwm-ext-large/resolve/main/pytorch_model.bin
!wget -nc -q -P {base_model_dir}/chinese-roberta-wwm-ext-large https://huggingface.co/hfl/chinese-roberta-wwm-ext-large/resolve/main/tokenizer.json
!wget -nc -q -P {base_model_dir}/chinese-hubert-base https://huggingface.co/TencentGameMate/chinese-hubert-base/resolve/main/config.json
!wget -nc -q -P {base_model_dir}/chinese-hubert-base https://huggingface.co/TencentGameMate/chinese-hubert-base/resolve/main/pytorch_model.bin
!wget -nc -q -P {base_model_dir}/chinese-hubert-base https://huggingface.co/TencentGameMate/chinese-hubert-base/resolve/main/preprocessor_config.json
!wget -nc -q -P {base_model_dir} https://huggingface.co/lj1995/GPT-SoVITS/resolve/main/s2G488k.pth
!wget -nc -q -P {base_model_dir} https://huggingface.co/lj1995/GPT-SoVITS/resolve/main/s2D488k.pth
!wget -nc -q -P {base_model_dir} https://huggingface.co/lj1995/GPT-SoVITS/resolve/main/s1bert25Hz-2kh-longer-epoch=68e-step=50232.ckpt

clear_output()
print("✅ Cài đặt hoàn tất!")
print("👉 Tiếp tục chạy Cell 2 để upload và xử lý audio.")

In [ ]:
# @title 2️⃣ Upload Audio + Cắt + ASR Tiếng Việt
# @markdown Chạy cell này để upload, cắt audio và tạo phụ đề tiếng Việt.
# @markdown ⏱️ Thời gian: ~5-15 phút (tùy độ dài audio)

import os
import shutil
from google.colab import files

# ========== CẤU HÌNH ==========
exp_name = "MyVoice" # @param {type:"string"}
# ==============================

# Tạo thư mục
input_folder = f"/content/dataset/{exp_name}"
sliced_folder = "output/slicer_opt"
asr_output = f"output/{exp_name}.list"

os.makedirs(input_folder, exist_ok=True)
os.makedirs(sliced_folder, exist_ok=True)

# Upload files
print("📤 Vui lòng upload file audio (.mp3, .wav):")
uploaded = files.upload()

for filename in uploaded.keys():
    shutil.move(filename, os.path.join(input_folder, filename))
    print(f"   ✓ {filename}")

# ========== BƯỚC 1: CẮT AUDIO ==========
print("\n" + "="*50)
print("🔪 BƯỚC 1: CẮT AUDIO THÀNH CÁC ĐOẠN NGẮN")
print("="*50)

!python tools/slice_audio.py "{input_folder}" "{sliced_folder}" -34 4000 300 10 500 0.9 0.25 0 1

sliced_files = [f for f in os.listdir(sliced_folder) if f.endswith(('.wav', '.mp3', '.flac'))]
print(f"\n✅ Đã cắt xong! Tổng cộng {len(sliced_files)} file audio.")

# ========== BƯỚC 2: ASR TIẾNG VIỆT ==========
print("\n" + "="*50)
print("🎤 BƯỚC 2: NHẬN DẠNG GIỌNG NÓI (ASR) TIẾNG VIỆT")
print("="*50)
print("⏳ Đang chạy Whisper large-v3... Vui lòng chờ.\n")

!python tools/asr/fasterwhisper_asr.py -i "{sliced_folder}" -o "output" -s large-v3 -l vi -p float16

# Tìm file .list được tạo
list_files = [f for f in os.listdir("output") if f.endswith('.list')]
if list_files:
    actual_list_file = f"output/{list_files[0]}"
    # Rename to exp_name.list for clarity
    if actual_list_file != asr_output:
        shutil.copy(actual_list_file, asr_output)
    
    with open(asr_output, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    
    print("\n" + "="*50)
    print("✅ HOÀN TẤT!")
    print("="*50)
    print(f"📄 File .list: {os.path.abspath(asr_output)}")
    print(f"📝 Tổng số dòng: {len(lines)}")
    print(f"\n📋 3 dòng đầu tiên:")
    for i, line in enumerate(lines[:3]):
        parts = line.strip().split('|')
        if len(parts) >= 4:
            print(f"   {i+1}. [{parts[2]}] {parts[3][:50]}...")
else:
    print("❌ Không tìm thấy file .list!")

print("\n👉 Tiếp tục chạy Cell 3 để mở WebUI.")
print(f"👉 Khi vào WebUI, điền đường dẫn này vào ô 'Text Label File':")
print(f"   {os.path.abspath(asr_output)}")

In [ ]:
# @title 3️⃣ Khởi động WebUI (Training + Inference)
# @markdown **Cell này chạy liên tục.** Mở link gradio.live để làm việc.
# @markdown
# @markdown **Trên WebUI:**
# @markdown - Tab 1A: Paste đường dẫn .list → Format data
# @markdown - Tab 1B: Train SoVITS → Train GPT
# @markdown - Tab 1C: Chọn model → Inference

import os

# Cấu hình paths
cwd = os.getcwd()
bert_path = os.path.join(cwd, "GPT_SoVITS/pretrained_models/chinese-roberta-wwm-ext-large")
hubert_path = os.path.join(cwd, "GPT_SoVITS/pretrained_models/chinese-hubert-base")

os.environ["cnhubert_base_path"] = hubert_path
os.environ["bert_path"] = bert_path
os.environ["is_share"] = "True"

print("🚀 Đang khởi động WebUI...")
print("⏳ Chờ 1-2 phút để load xong.")
print("\n" + "="*50)
print("📋 HƯỚNG DẪN SỬ DỤNG WEBUI:")
print("="*50)
print("1. Tab 1 → 1A: Điền tên experiment và đường dẫn file .list")
print("2. Bấm nút 'One-click format'")
print("3. Tab 1 → 1B: Train SoVITS (epochs=8), sau đó Train GPT (epochs=15)")
print("4. Tab 1 → 1C: Chọn model → Inference")
print("="*50 + "\n")

!sed -i 's/is_share = False/is_share = True/g' config.py 2>/dev/null || true
!python webui.py --share

In [ ]:
# @title 4️⃣ Lưu Model về Google Drive (Tùy chọn)
# @markdown Chạy cell này SAU KHI train xong để backup model.

from google.colab import drive
import shutil
import os

exp_name = "MyVoice" # @param {type:"string"}
drive_path = f"/content/drive/MyDrive/GPT_SoVITS_Models/{exp_name}"

# Mount Drive
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

os.makedirs(drive_path, exist_ok=True)

print(f"📦 Đang lưu model {exp_name} vào Drive...")

# Copy trained models
!cp -r logs/{exp_name}/* "{drive_path}/" 2>/dev/null || true
!cp GPT_SoVITS/weights/*.pth "{drive_path}/" 2>/dev/null || true
!cp GPT_SoVITS/weights/*.ckpt "{drive_path}/" 2>/dev/null || true

print(f"\n✅ Đã lưu xong tại: {drive_path}")
print("📂 Các file đã lưu:")
!ls -la "{drive_path}"